In [1]:
#Toxicity Prediction
#Author : Shubham Deshmukh 
#Group : [Maple Squad]
#Participants : Jaskiran Kaur, Sohini Sarkar, Shubham Deshmukh
#Email : x2020gfv@stfx.ca
#Student No. : 202006307

In [2]:
#KAGGLE PRIVATE SCORE: 0.80294
#F1 MACRO :            0.8085145591717199
#Varince0.02 -> RFE 200 XGB default->RFECV GBC Default ->sampling-> scaling

In [19]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import RFE, RFECV , SelectFromModel
from sklearn.feature_selection import SelectKBest,f_classif


from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

from xgboost.sklearn import XGBClassifier


In [20]:

#importing dataset CSV files
url_train_datafile = 'train.csv'
url_test_datafile = 'test.csv'
url_feamat_datafile = 'feamat.csv'
url_mappings_datafile = 'features_id_name_mappings.csv' # We Dont have to Import this file as CSV for now


train_rawdata = pd.read_csv(url_train_datafile)
test_rawdata = pd.read_csv(url_test_datafile)
feamat_rawdata = pd.read_csv(url_feamat_datafile)

tempTrain =train_rawdata["Id"].str.split(";", n = 1, expand = True) 

train_rawdata["AssayId"]= tempTrain[1] 
train_rawdata["Id"]= tempTrain[0]

tempTest =test_rawdata["x"].str.split(";", n = 1, expand = True) 

test_rawdata["AssayId"]= tempTest[1] 
test_rawdata["Id"]= tempTest[0]
test_rawdata =  test_rawdata.drop(['x'],axis=1)


# Removing the NA, Null and Infinty Values in Feature Matrix with mean Values
feamat_rawdata = feamat_rawdata.replace([np.inf, -np.inf], np.nan)


# train_rawdata , test_rawdata are our main Test Train dataset varibles 

newMergedDF = pd.merge(train_rawdata, feamat_rawdata,left_on='Id', right_on='V1', how = "left")

#Replacing NOCAS_ with constant number(999) and removing -*
newMergedDF[['V1','Id']] = newMergedDF[['V1','Id']].replace('NOCAS_','999', regex=True)
newMergedDF[['V1','Id']] = newMergedDF[['V1','Id']].replace('-*','',regex=True)
newMergedDF =  newMergedDF.drop(['V1'],axis=1)#,'V2,'Id'
#newMergedDF is our new merged DataSet ()


newMergedTestDF = pd.merge(test_rawdata, feamat_rawdata,left_on='Id', right_on='V1', how = "left")

newMergedTestDF[['V1','Id']] = newMergedTestDF[['V1','Id']].replace('NOCAS_','999', regex=True)
newMergedTestDF[['V1','Id']] = newMergedTestDF[['V1','Id']].replace('-*','',regex=True)
newMergedTestDF =  newMergedTestDF.drop(['V1'],axis=1)#,'V2,'Id'


print('-----------------1- DATA PREPARED SUCCESSFULLY-----------------')



-----------------1- DATA PREPARED SUCCESSFULLY-----------------


In [5]:
columns_toDrop = [ 'Expected','V29','V30']#,'V29','V30'
columns_toDropTest = ['V29','V30']#'V29','V30'
#,'V33','V36','V49','V31','V35','V38','V45','V67'

Y_train = newMergedDF[['Expected']].astype(int)
X_train = newMergedDF.drop(columns_toDrop,axis=1)
X_test = newMergedTestDF.drop(columns_toDropTest,axis=1)

    

imp = KNNImputer(n_neighbors=2, weights="uniform")
data = X_train
imp = imp.fit(data)      
data = imp.transform(data) 
X_train = pd.DataFrame(data)   



imp = KNNImputer(n_neighbors=2, weights="uniform")
data = X_test
imp = imp.fit(data)      
data = imp.transform(data) 
X_test = pd.DataFrame(data)   



print('-----------------2- DATA SPLITTED SUCCESSFULLY-----------------')



-----------------2- DATA SPLITTED SUCCESSFULLY-----------------


In [6]:
# FEATURE SELECTION PART 1

#APPLY VARIANCE THRESHOLD TO REMOVE SOME LABELS
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(0.02)
selector.fit(X_train)

X_train= selector.transform(X_train)
X_test= selector.transform(X_test)

X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

#print(X_train)
#print(X_test)

print('-----------------3- VARIANCE THRESHOLD FEATURE SELECTION 1------------------')

-----------------3- VARIANCE THRESHOLD FEATURE SELECTION 1------------------


In [7]:
# FEATURE SELCTION PART 2

#RFE DONE TO SELECT 200 LABLES AFTER VARINCE 

X_train1 = X_train
Y_train1 = Y_train

# define RFE

model = XGBClassifier(random_state = 0)

rfe = RFE(estimator=model,n_features_to_select=200, step = 0.30)
rfe.fit(X_train1, Y_train1)

X_train = pd.DataFrame(rfe.transform(X_train))
X_test = pd.DataFrame(rfe.transform(X_test))

#print(X_train)
#print(X_test)

print('-----------------4- RFE FEATURE SELECTION 2------------------')



C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[03:42:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[03:43:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[03:44:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[03:45:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
-----------------4- RFE FEATURE SELECTION 2------------------


In [8]:
# FEATURE SELCTION PART 3

#RFECV DONE TO SELECT LABLES AFTER RFE 


X_train1 = X_train
Y_train1 = Y_train

rfecv = RFECV(cv=5,estimator=GradientBoostingClassifier(random_state=0),min_features_to_select=30, scoring='f1_macro', step=0.1)
rfecv.fit(X_train1, Y_train1)

X_train = pd.DataFrame(rfecv.transform(X_train))
X_test = pd.DataFrame(rfecv.transform(X_test))


print('-----------------5- RFECV FEATURE SELECTION 3------------------')



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

-----------------5- RFECV FEATURE SELECTION 3------------------


In [9]:
#DO NOT USE THIS- ONLY FOR TESTING
#FEATURE SELCTION PART 4

#SELECTKBEST ANNOVA DONE TO SELECT 40 LABLES AFTER RFECV


#skbest = SelectKBest(f_classif, k=40).fit(X_train, Y_train)
#X_train = skbest.transform(X_train)
#X_test = skbest.transform(X_test)


#print('-----------------6- SELECTKBEST FEATURE SELECTION 4------------------')


In [10]:
#CHECK THE DIMENSION OF THE DATA HERE 

#X_train = pd.DataFrame(X_train)
#X_test = pd.DataFrame(X_test)

#print(X_train)
#print(X_train)

In [11]:
#INTERNAL VALIDATION OF THE MODEL
#F1 MACRO SCORE -- RANDOMOVER SAMPLING AND SCALING
 


train_X, val_X, train_y, val_y = train_test_split(X_train, Y_train, random_state=0,test_size=0.2)


ROS = RandomOverSampler(random_state=42)
train_X,train_y = ROS.fit_resample(train_X,train_y)


feature_scaler = StandardScaler()
train_X = feature_scaler.fit_transform(train_X)
val_X = feature_scaler.transform(val_X)


model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                          colsample_bynode=1, colsample_bytree=0.9293233097407273,
                          gamma=0.22110176557762784, gpu_id=-1, importance_type='gain',
                          interaction_constraints='', learning_rate=0.05, #0.1510928495444365,
                          max_delta_step=1, max_depth=14, min_child_weight=1,
                          monotone_constraints='()', n_estimators=600, n_jobs=-1,
                          num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
                          scale_pos_weight=1, subsample=0.779274053965761,
                          tree_method='exact', validate_parameters=1, verbosity=None)

model = model.fit(train_X, train_y)

y_pred = model.predict(val_X)


macro_score = f1_score(val_y, y_pred, average='macro')
print('F1 Macro Score:',macro_score)

print('-----------------7- INTERNAL VALIDATION ------------------')


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[04:19:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
F1 Macro Score: 0.8085145591717199
-----------------7- INTERNAL VALIDATION ------------------


In [12]:
#RANDOM OVER SAMPLING DONE ON THE ENTIRE TRAINING DATA


ovs = RandomOverSampler(random_state=42)
X_train,Y_train = ovs.fit_resample(X_train,Y_train)


print('-----------------8- SAMPLING DONE SUCESSFULLY------------------')


-----------------8- SAMPLING DONE SUCESSFULLY------------------


In [13]:
#STANDARD SCALING DONE ON THE ENTIRE TRAINING DATA

feature_scaler = StandardScaler()
X_train = feature_scaler.fit_transform(X_train)
X_test = feature_scaler.transform(X_test)

print('-----------------9- SCALING DONE SUCESSFULLY------------------')



-----------------9- SCALING DONE SUCESSFULLY------------------


In [14]:
#DATA LOADING AND PREDICTIONS ON THE MODEL


XGB = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                          colsample_bynode=1, colsample_bytree=0.9293233097407273,
                          gamma=0.22110176557762784, gpu_id=-1, importance_type='gain',
                          interaction_constraints='', learning_rate=0.05, #0.1510928495444365,
                          max_delta_step=1, max_depth=14, min_child_weight=1,
                          monotone_constraints='()', n_estimators=600, n_jobs=-1,
                          num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
                          scale_pos_weight=1, subsample=0.779274053965761,
                          tree_method='exact', validate_parameters=1, verbosity=None).fit(X_train,Y_train)
Y_prediction = XGB.predict(X_test) 



print('-----------------10- MODEL FITTING AND PREDICTING DONE SUCESSFULLY------------------')

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[04:22:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
-----------------10- MODEL FITTING AND PREDICTING DONE SUCESSFULLY------------------


In [15]:
#EXPORT THE EXCEL FILE OF THE PREDICTION

finalExpected = pd.DataFrame(Y_prediction, columns = ['Predicted'])

test_rawdata1 = pd.read_csv(url_test_datafile)

finalResultConcated =  pd.concat([test_rawdata1, finalExpected],axis=1)
finalResultConcated.rename(columns = {'x':'Id'}, inplace = True)

finalResultConcated.to_csv('Maple_Squad-1.csv',index=False)
print(test_rawdata1.info())
print(finalExpected.info())
print(finalResultConcated.info())



print('-----------------11- PREDICTION RESULT EXCEL EXPORTED SUCESSFULLY------------------')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11139 entries, 0 to 11138
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   x       11139 non-null  object
dtypes: object(1)
memory usage: 87.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11139 entries, 0 to 11138
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   Predicted  11139 non-null  int32
dtypes: int32(1)
memory usage: 43.6 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11139 entries, 0 to 11138
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Id         11139 non-null  object
 1   Predicted  11139 non-null  int32 
dtypes: int32(1), object(1)
memory usage: 130.7+ KB
None
-----------------11- PREDICTION RESULT EXCEL EXPORTED SUCESSFULLY------------------


In [16]:

#USED FOR HYPERPARAMETER TUNING OF XGB ONLY ONCE TO GET THE BEST PARAMETER 

#----------------------DO NOT RUN THIS---ITS A ONE TIME CODE-------------------

#----------------------TIME-TAKEN: 2 DAYS -------------------------------------


#from sklearn.model_selection import train_test_split, RandomizedSearchCV
#def build_xgboost(X_train, y_train, X_test, y_test, n_iter):
#    from scipy.stats import randint, uniform
#    xgb_param_fixed = {        
#        'max_delta_step': 1,
#        'n_jobs': -1,
#        'n_estimators': 300,
#        'learning_rate': 0.1}
#    xgb_base = XGBClassifier(**xgb_param_fixed)

#    xgb_param_options = {
#        'max_depth': randint(low=3, high=15),
#        'colsample_bytree': uniform(loc=0.7, scale=0.3),
#        'subsample': uniform(loc=0.7, scale=0.3),
#        'learning_rate':uniform(loc=0.05, scale=0.30),
#        'min_child_weight':randint(low=1, high=7),
#        'gamma':uniform(loc=0.0, scale=0.4) }
    
#    eval_set = [(X_train, y_train), (X_test, y_test)]
#    xgb_fit_params = {   
#        'eval_metric': 'auc', 
#        'eval_set': eval_set,
#        'early_stopping_rounds': 5,
#        'verbose': False
#    }

#   model_xgb = RandomizedSearchCV(
#        estimator=xgb_base,
#        param_distributions=xgb_param_options,
#        cv=10,   
#        n_iter=n_iter,
#        n_jobs=-1,
#        verbose=1
#    ).fit(X_train, y_train, **xgb_fit_params)
    
#    print('Best score obtained: {0}'.format(model_xgb.best_score_))
#    print('Best Parameters:')
#    for param, value in model_xgb.best_params_.items():
#        print('\t{}: {}'.format(param, value))

#    return model_xgb.best_estimator_

#train_X, val_X, train_y, val_y = train_test_split(X_train, Y_train, random_state=42)

#xgb_model = build_xgboost(train_X, train_y, val_X, val_y, n_iter=15)
#ntree_limit = xgb_model.best_ntree_limit
#y_pred_prob = xgb_model.predict_proba(val_X, ntree_limit=ntree_limit)[:, 1]
#print('auc:', roc_auc_score(val_y, y_pred_prob))+*9

#----------OUTPUT--------------------
#Best Parameters:
#	colsample_bytree: 0.9293233097407273
#	gamma: 0.22110176557762784
#	learning_rate: 0.1510928495444365
#	max_depth: 14
#	min_child_weight: 1
#	subsample: 0.779274053965761

#XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#              colsample_bynode=1, colsample_bytree=0.9293233097407273,
#              gamma=0.22110176557762784, gpu_id=-1, importance_type='gain',
#              interaction_constraints='', learning_rate=0.1510928495444365,
#              max_delta_step=1, max_depth=14, min_child_weight=1, missing=nan,
#              monotone_constraints='()', n_estimators=300, n_jobs=-1,
#              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
#              scale_pos_weight=1, subsample=0.779274053965761,
#              tree_method='exact', validate_parameters=1, verbosity=None)

In [17]:
#----------------------------- CODE ENDS HERE--------------------------- 


#------------------------BELOW CODE ONLY USED FOR TESTING-------------------------

In [18]:
#ONLY FOR TESTING PURPOSES 


#X_train = pd.DataFrame(X_train)
#importances = pd.DataFrame({'feature':X_train.columns,'importance':np.round(XGB.feature_importances_,3)})
#importances = importances.sort_values('importance',ascending=True).set_index('feature')
#print(importances.head(60))
